# Generator Coroutines

This notebook talks about old style coroutines that do not use the `async/await` syntax. These old style coroutines are very similar to generators, in fact their type is a `generator`. The only difference is that in addition to yielding values, co-routines can also accept parameters with yield. So it is a weird function which can accept params in the middle of its execution and act on it.

David Beazley's [A Curious Course on Coroutines and Concurrency](http://www.dabeaz.com/coroutines/) [[downloaded pdf](./Coroutines.pdf)]. Like all of his courses, it seems good at first, but after I have thought about it in great detail, I don't like it anymore.

In [62]:
from cprint import cprint
import random

## Execution Flow
First question is how does one get an argument to the coroutine mid-execution? This is done with the help of the `send` function that is available on all `generators`. Remember how for plain generators, calling `next(gen)` results in the function executing till it hits a `yield val` or `yield` statement. The return value of `next` is then either `None` or `val`. Think of `next` as syntactic sugar on top of `send`. `v = gen.send(None)` is equivalent to `v = next(gen)`. 

In [2]:
def coro_1(x):
    cprint(0, "I am doing")
    cprint(0, "something of great")
    cprint(0, "importance")
    val = x + 10
    arg = yield val
    cprint(0, f"Got {arg} from caller.")
    yield arg + 10
    cprint(0, "I am done.")
    return True

In [3]:
coro_1_obj_1 = coro_1(3)
type(coro_1_obj_1)

generator

Now the first time I call `coro_1_obj_1` it will execute till line #6. But only partially, calling `coro_1_obj_1.send(None)` will return 13. The call is now suspended on line #6. There is nothing assigned to `arg` yet. If I call `coro_1_obj_1.send(5)` now, the exeuction resumes from line #6 with `arg` being assigned the value 5. The call then executes till line #8, where the control is yielded back to the caller.

In [4]:
next(coro_1_obj_1)

I am doing
something of great
importance


13

In [5]:
coro_1_obj_1.send(5)

Got 5 from caller.


15

Whenever I see the following pattern -

```python
x = yield expr
```

Remember, `x` has got **nothing to do with** `expr`! `x` will not take the value "yield"ed by `expr`. These are two completely different things. The value of `expr` goes to the caller of `v = next(g)` or alternatively `v = g.send(...)`. So `v` will get the return value of `expr`. But `x` will get whatever the caller will send it, i.e., `g.send(val_x)` will result in `x` being assinged `val_x`. According to dbeazley it is best to keep coroutines and generators separate, i.e., have either of the two patterns, but not the one above.

```python
def generator():
    ...
    yield val
    ...

def coroutine():
    ...
    arg = yield
    ...
```

Mixing the two can be confusing.

## Send Rules
### Run Till `yield`
One pitfall in this programming model is that often times I forget that `coro_1_obj_1.send(v)` will run till the next `yield`. It is not just assinging the value to `arg` and waiting on line #6. It will run until it finds another `yield` on line #8. If it does not find another `yield`, it will raise a `StopIteration` error just like a plain generator.

In [6]:
coro_1_obj_1.send(True)

I am done.


StopIteration: True

### Priming

It does not make a lot of sense to send `coro_1_obj_1` a value immediately after creating it because there is nothing to recieve the value. This why these coroutines have to "primed" first by sending a `None` so they can reach the first `yield` statement and then wait for the argument from the caller. While this makes perfect sense from the implementation, the semantics become a bit weird. The first time around, the only valid value to send is `None`, anything else will result in an error.

In [7]:
coro_1_obj_2 = coro_1(3)
coro_1_obj_2.send(10)

TypeError: can't send non-None value to a just-started generator

### Subsequent `send` Calls
After the first call, all other `send` calls can send in whatever they want. Even if the coroutine is not accepting any arguments, the caller can send values and they will be silently dropped on the floor.

In [15]:
def coro_2(x):
    cprint(0, "I am doing")
    cprint(0, "something of great")
    cprint(0, "importance")
    val = x + 10
    arg = yield val
    cprint(0, f"Got {arg} from caller.")
    yield arg + 10
    cprint(0, "Continuing on my merry journey ignoring anything you might have sent me.")
    yield "HAHA"
    cprint(0, "And now I am done")
    return True

In [16]:
coro_2_obj_1 = coro_2(3)

# Prime the coroutine
val = next(coro_2_obj_1)  
print("Primed val: ", val)

val = coro_2_obj_1.send(5)
print("Got val: ", val)

val = coro_2_obj_1.send("rubbish")
print("Got val: ", val)

I am doing
something of great
importance
Primed val:  13
Got 5 from caller.
Got val:  15
Continuing on my merry journey ignoring anything you might have sent me.
Got val:  HAHA


## Closing
This applies to plain old generators as well as coroutines. After `gen.close()` is called, regardless of how many more `yield`s are remaining in the function, the next call to `next` will result in a `StopIteration` error. `gen.close()` will return always return `None`.


In [32]:
def coro_3():
    yield "first"
    yield "second"
    yield "third"
    cprint(0, "I am done.")
    return True

For `coro_3` , if I call `close` after calling `next` twice, then even though a thrid `yield` is still there, I'll hit a `StopIteration`. Lines #4 to #6 never get executed.

In [33]:
coro_3_obj_1 = coro_3()
print(next(coro_3_obj_1))
print(next(coro_3_obj_1))

first
second


In [53]:
ret = coro_3_obj_1.close()
print(ret)

None


In [35]:
next(coro_3_obj_1)

StopIteration: 

Even if I call `close` after all the `yield`s are done, lines #5 and #6 never get executed.

In [36]:
coro_3_obj_2 = coro_3()
print(next(coro_3_obj_2))
print(next(coro_3_obj_2))
print(next(coro_3_obj_2))
coro_3_obj_2.close()
print(next(coro_3_obj_2))

first
second
third


StopIteration: 

The only way to make lines #5 and #6 execute is to exhaust the generator and hit `StopIteration` naturally.

In [44]:
try:
    coro_3_obj_3 = coro_3()
    print(next(coro_3_obj_3))
    print(next(coro_3_obj_3))
    print(next(coro_3_obj_3))
    print(next(coro_3_obj_3))
except StopIteration as serr:
    ret = serr.value
    print("Generator returned ", ret)

first
second
third
I am done.
Generator returned  True


### Handling `GeneratorExit` Exception
One way to ensure that some clean up code is executed when `close` is called is to add it as part of `GeneratorExit` exception handling code block. Whenever `close` is called, this exception is raised in the generator and handled if the handler is there. Otherwise it is silently ignored. But even in this case, there is no point in the generator actually returning anything. The return value is dropped.

In [49]:
def coro_4():
    try:
        yield "first"
        yield "second"
        yield "third"        
    except GeneratorExit:
        cprint(0, "I am done.")
        return 42  # This is pointless, nothing is returned.

In [54]:
coro_4_obj_1 = coro_4()
print(next(coro_4_obj_1))
print(next(coro_4_obj_1))
print(coro_4_obj_1.close())

first
second
I am done.
None


## Sub Generators

Based on what we have seen so far, it is not possible to have nested yields, i.e., I cannot have a function yield the value of another function that is also yielding values.

In [55]:
def add(x, y):
    yield x + y


def calc():
    val = yield add(2, 2)
    print(val)
    yield

In [56]:
c = calc()
next(c)

<generator object add at 0x1096255b0>

I have to resort to the so-called "trampoline" trick.

In [57]:
c = calc()
sub = next(c)  # sub now has the add generator
val = next(sub)  # val has the value yielded by add
c.send(val)  # lets send that value back to calc

4


To avoid this sort of weird calling pattern, [PEP 380](https://peps.python.org/pep-0380/) introduced the `yield from` syntax. `x = yield from subroutine(...)` is semantically better than `x = yield expr` because in the `yield from` syntax, the value yielded by the `subroutine` is actually assigned to `x`. There is no `send`ing happeing from the caller's side to send values to `x`.

In [58]:
def calc2():
    val = yield from add(2, 2)
    print(val)
    yield

In [59]:
c = calc2()
next(c)

4

Of course this only works when the subroutine is also a generator that yields values instead of a plain old function that returns them.

In [60]:
def add2(x, y):
    return x + y

def calc3():
    val = yield from add2(2, 2)
    print(val)
    yield

In [61]:
c = calc3()
next(c)

TypeError: 'int' object is not iterable

While `val = yield from subroutine(...)` is the most common pattern of using `yield from`, it is also possible to have `yield from subroutine(...)` without assigning its value to `val`. In this case, the parent generator will simply act as a pass-through to yield values from the sub generator. It will wait until the sub generator is exhausted before it moves on to the next line. In the `gen_two` below, the code execution will never move beyond line #7 because `gen_one` will never be exhausted. Everytime I call `next(gen2)` it will execute line #7 by pulling a value yielded by `gen_one` and yield that.

In [91]:
def gen_one():
    while True:
        yield random.randint(10, 1000)
    
def gen_two():
    cprint(1, "Starting gen_two")
    yield from gen_one()
    cprint(1, "Finished yielding from gen_one")
    yield random.random()

In [92]:
# for x in gen_two():
#     print(x)
gen2 = gen_two()
for _ in range(3):
    print(next(gen2))

Starting gen_two
231
747
95
